In [ ]:
#Pour lire un fichier CSV et le charger dans un DataFrame Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL Students").getOrCreate()

# Charger le fichier CSV
df = spark.read.csv("/content/drive/MyDrive/stdnt/etudiants_third.csv", header=True, inferSchema=True)
df.show()

+--------+---------+---+------------+
|     nom|   prenom|age|     filiere|
+--------+---------+---+------------+
|   Black| Jennifer| 18|Data Science|
| Ramirez|  Jeffrey| 20|     Gestion|
|  Ingram|    Wanda| 18|     Gestion|
|   Ramos|    Wanda| 30|Data Science|
|   Lopez| Kathleen| 28|   Marketing|
|   Perez|   Hannah| 19|     Finance|
|  Cortez|    Tammy| 30|Data Science|
|   Hardy|    Laura| 20|     Gestion|
|   Welch|    Amber| 23|   Marketing|
|Campbell|    Kevin| 25|Data Science|
|  Wilson|  Kristin| 20|     Gestion|
|  Fisher|Christine| 28|     Gestion|
|  Miller|  Shannon| 28|Informatique|
|    Hull|    David| 22|     Finance|
|Anderson|    Randy| 25|Informatique|
|  Monroe|   Sheryl| 25|   Marketing|
|   Evans|   Andrew| 29|   Marketing|
|    Cook|    Donna| 26|Data Science|
|   Munoz|    Tyler| 28|     Finance|
|  Porter|     Luke| 19|     Finance|
+--------+---------+---+------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import upper, col  # Import upper and col functions

# Convertir les noms en majuscules
df1 = df.withColumn("nom", upper(col("nom")))
df1.show()

+--------+---------+---+------------+
|     nom|   prenom|age|     filiere|
+--------+---------+---+------------+
|   BLACK| Jennifer| 18|Data Science|
| RAMIREZ|  Jeffrey| 20|     Gestion|
|  INGRAM|    Wanda| 18|     Gestion|
|   RAMOS|    Wanda| 30|Data Science|
|   LOPEZ| Kathleen| 28|   Marketing|
|   PEREZ|   Hannah| 19|     Finance|
|  CORTEZ|    Tammy| 30|Data Science|
|   HARDY|    Laura| 20|     Gestion|
|   WELCH|    Amber| 23|   Marketing|
|CAMPBELL|    Kevin| 25|Data Science|
|  WILSON|  Kristin| 20|     Gestion|
|  FISHER|Christine| 28|     Gestion|
|  MILLER|  Shannon| 28|Informatique|
|    HULL|    David| 22|     Finance|
|ANDERSON|    Randy| 25|Informatique|
|  MONROE|   Sheryl| 25|   Marketing|
|   EVANS|   Andrew| 29|   Marketing|
|    COOK|    Donna| 26|Data Science|
|   MUNOZ|    Tyler| 28|     Finance|
|  PORTER|     Luke| 19|     Finance|
+--------+---------+---+------------+
only showing top 20 rows



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Filtrer les étudiants de plus de 20 ans
df2= df1.filter(df1["age"] > 20)
df2.show()

+--------+---------+---+------------+
|     nom|   prenom|age|     filiere|
+--------+---------+---+------------+
|   RAMOS|    Wanda| 30|Data Science|
|   LOPEZ| Kathleen| 28|   Marketing|
|  CORTEZ|    Tammy| 30|Data Science|
|   WELCH|    Amber| 23|   Marketing|
|CAMPBELL|    Kevin| 25|Data Science|
|  FISHER|Christine| 28|     Gestion|
|  MILLER|  Shannon| 28|Informatique|
|    HULL|    David| 22|     Finance|
|ANDERSON|    Randy| 25|Informatique|
|  MONROE|   Sheryl| 25|   Marketing|
|   EVANS|   Andrew| 29|   Marketing|
|    COOK|    Donna| 26|Data Science|
|   MUNOZ|    Tyler| 28|     Finance|
|     COX| Gabriela| 24|Data Science|
|     FOX|  William| 30|Data Science|
| DANIELS|Christian| 26|   Marketing|
| GARRETT|   Thomas| 27|      Design|
|   ALLEN| Benjamin| 25|Informatique|
|   PRICE|    Molly| 25|     Gestion|
| COLLINS| Nicholas| 29|     Finance|
+--------+---------+---+------------+
only showing top 20 rows



In [ ]:
#Calculer la moyenne d'âge par filière
from pyspark.sql.functions import avg
stats = df2.groupBy("Filiere").agg(avg("age").alias("moyenne_age"))
stats.show()

+------------+------------------+
|     Filiere|       moyenne_age|
+------------+------------------+
|Informatique|26.363636363636363|
|     Gestion|25.956521739130434|
|     Finance|25.136363636363637|
|   Marketing|25.941176470588236|
|      Design|25.666666666666668|
|Data Science| 26.88888888888889|
+------------+------------------+



In [ ]:
#Sauvegarde les données transformées dans un fichier CSV
df2.write.csv("/content/etudiants_transforme.csv", header=True)

In [ ]:
!pip install kafka-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 9.1 MB/s eta 0:00:00


In [ ]:
import csv
import json
from kafka import KafkaProducer

# Configuration du producteur Kafka
producer = KafkaProducer(
    bootstrap_servers='kafka-23713192-ilyassayh222-2b4c.h.aivencloud.com:16372',
    security_protocol='SSL',
    ssl_cafile='/content/drive/MyDrive/stdnt/ca.pem',
    ssl_certfile='/content/drive/MyDrive/stdnt/service.cert',
    ssl_keyfile='/content/drive/MyDrive/stdnt/service.key',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Convertir les données en JSON
)

# Chemin du fichier CSV
csv_file_path = "/content/drive/MyDrive/stdnt/etudiants_third.csv"

# Lire le fichier CSV et envoyer chaque ligne à Kafka
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)  # Lire les lignes sous forme de dictionnaire
    for row in csv_reader:
        producer.send('students', value=row)  # Envoyer la ligne comme message JSON
        print(f"Message envoyé : {row}")

# Finaliser l'envoi
producer.flush()
print("Tous les messages ont été envoyés !")

Message envoyé : {'nom': 'Black', 'prenom': 'Jennifer', 'age': '18', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Ramirez', 'prenom': 'Jeffrey', 'age': '20', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Ingram', 'prenom': 'Wanda', 'age': '18', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Ramos', 'prenom': 'Wanda', 'age': '30', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Lopez', 'prenom': 'Kathleen', 'age': '28', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Perez', 'prenom': 'Hannah', 'age': '19', 'filiere': 'Finance'}
Message envoyé : {'nom': 'Cortez', 'prenom': 'Tammy', 'age': '30', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Hardy', 'prenom': 'Laura', 'age': '20', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Welch', 'prenom': 'Amber', 'age': '23', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Campbell', 'prenom': 'Kevin', 'age': '25', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Wilson', 'prenom': 'Kristin', 'age': '20', 'filiere': 

In [ ]:
!pip install confluent-kafka


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 69.9 MB/s eta 0:00:00


In [ ]:
import csv
import json
from kafka import KafkaConsumer
from collections import defaultdict

# Configuration du consommateur Kafka
consumer = KafkaConsumer(
    'students',
    bootstrap_servers='kafka-23713192-ilyassayh222-2b4c.h.aivencloud.com:16372',
    security_protocol='SSL',
    ssl_cafile='/content/drive/MyDrive/stdnt/ca.pem',
    ssl_certfile='/content/drive/MyDrive/stdnt/service.cert',
    ssl_keyfile='/content/drive/MyDrive/stdnt/service.key',
    group_id='CONSUMER_GROUP_ID',
    auto_offset_reset='earliest',  # Lire les messages depuis le début si nécessaire
    value_deserializer=lambda v: json.loads(v.decode('utf-8'))  # Décoder les messages JSON
)

# Liste pour stocker les données transformées
transformed_data = []

try:
    # Boucle pour lire les messages en continu
    for msg in consumer:
        message = msg.value  # Récupérer la valeur du message
        print(f"Message reçu : {message}")

        try:
            # Transformation des données reçues
            nom = message['Nom'].strip().upper()          # Nom en majuscules
            prenom = message['Prénom'].strip().upper()    # Prénom en majuscules
            age = int(message['Âge'])                    # Conversion de l'âge en entier
            filiere = message['Filière'].strip()         # Nettoyer la filière

            # Filtrage : étudiants de plus de 20 ans
            if age > 20:
                transformed_data.append({
                    'Nom': nom,
                    'Prénom': prenom,
                    'Âge': age,
                    'Filière': filiere
                })
        except KeyError as e:
            print(f"Clé manquante : {e}. Message ignoré.")
        except ValueError:
            print("Erreur de formatage des données reçues. Message ignoré.")
except KeyboardInterrupt:
    print("\nArrêt du consommateur.")
finally:
    consumer.close()

# Calcul des statistiques : Moyenne d'âge par filière
stats = defaultdict(list)
for student in transformed_data:
    stats[student['Filière']].append(student['Âge'])

average_age_by_field = {
    field: sum(ages) / len(ages) for field, ages in stats.items()
}

# Affichage des statistiques calculées
print("\nStatistiques : Moyenne d'âge par filière")
for filiere, avg_age in average_age_by_field.items():
    print(f"Filière: {filiere}, Âge moyen: {avg_age:.2f}")

# Sauvegarde des données transformées dans un fichier CSV
output_file = "etudiants_transformed.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Nom', 'Prénom', 'Âge', 'Filière'])
    writer.writeheader()
    writer.writerows(transformed_data)

print(f"\nDonnées transformées sauvegardées dans {output_file}")


Message reçu : {'key': 'test', 'value': 'Hello Kafka!'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Black', 'prenom': 'Jennifer', 'age': '18', 'filiere': 'Data Science'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Ramirez', 'prenom': 'Jeffrey', 'age': '20', 'filiere': 'Gestion'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Ingram', 'prenom': 'Wanda', 'age': '18', 'filiere': 'Gestion'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Ramos', 'prenom': 'Wanda', 'age': '30', 'filiere': 'Data Science'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Lopez', 'prenom': 'Kathleen', 'age': '28', 'filiere': 'Marketing'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Perez', 'prenom': 'Hannah', 'age': '19', 'filiere': 'Finance'}
Clé manquante : 'Nom'. Message ignoré.
Message reçu : {'nom': 'Cortez', 'prenom': 'Tammy', 'age': '30', 'filiere': 'Data Science'}
Clé manquante : 'Nom'. Message ignoré.


ERROR:kafka.consumer.fetcher:Fetch to node 2 failed: Cancelled: <BrokerConnection node_id=2 host=18.192.106.31:16372 <connected> [IPv4 ('18.192.106.31', 16372)]>



Arrêt du consommateur.

Statistiques : Moyenne d'âge par filière

Données transformées sauvegardées dans etudiants_transformed.csv
